En este archivo, voy a desarrollar la función que se me pide en el segundo endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_2.png "Title")

Importación de las librerías necesarias para trabajar

In [1]:
import pandas as pd

Lectura de los conjuntos de datos necesarios necesarios y breve vistazo de la cabecera de los mismos

In [2]:
user_reviews = pd.read_parquet("./EDA/user_reviews_complete.parquet")
user_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [3]:
user_items = pd.read_parquet("../EDA/user_items_complete.parquet")
user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [4]:
steam_games = pd.read_parquet("../Datasets/steam_games_post_EDA.parquet")
steam_games.head()

,id,app_name,developer,price,release_year,categorical
88310,761140,Lost Summoner Kitty,Kotoshiro,4.99,2018,"[Action, Casual, Indie, Simulation, Strategy, ..."
88311,643980,Ironbound,Secret Level SRL,0.00,2018,"[Free to Play, Indie, RPG, Strategy, Card Game..."
88312,670290,Real Pool 3D - Poolians,Poolians.com,0.00,2017,"[Casual, Free to Play, Indie, Simulation, Spor..."
88313,767400,弹炸人2222,彼岸领域,0.99,2017,"[Action, Adventure, Casual, Single-player]"
88315,772540,Battle Royale Trainer,Trickjump Games Ltd,3.99,2018,"[Action, Adventure, Simulation, FPS, Shooter, ..."


A partir de estos Dataframe, analizo lo que me piden:
- A partir del id de un usuario, devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación, y la cantidad de items que tiene el usuario. Emtonces lo que necesito es:
    - id de un usuario
    - cantidad de dinero gastado por el usuario
    - porcentaje de recomendación
    - cantidad de items que tiene el usuario

Veamos las columnas que tenemos disponibles en todos nuestros conjuntos de datos.

In [5]:
print(user_reviews.columns)
print()
print(user_items.columns)
print()
print(steam_games.columns)
print()

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id'],
      dtype='object')

Index(['id', 'app_name', 'developer', 'price', 'release_year', 'categorical'], dtype='object')



De acuerdo a estos resultados...
Las columnas que voy a usar para este trabajo son:
- user_reviews: user_id, recommend
- user_items: user_id, item_id
- steam_games: id = item_id, price.

El resto de columnas no me sirve, así que las descarto

In [6]:
for column in user_reviews.columns: # Por cada columna en el dataframe user_reviews
    if column not in ['user_id', 'recommend']: # Si la columna no es la que me sirve
        user_reviews.drop(column, axis=1, inplace=True) # La elimino
print(user_reviews.columns) # Muestro las columnas que me quedaron cómo forma de comprobar los resultados

# Repito lo mismo con los otros dos dataframe

for column in user_items.columns:
    if column not in ['user_id', 'item_id']:
        user_items.drop(column, axis=1, inplace=True)
print(user_items.columns)

for column in steam_games.columns:
    if column not in ['price', 'id']:
        steam_games.drop(column, axis=1, inplace=True)
print(steam_games.columns)


Index(['recommend', 'user_id'], dtype='object')
Index(['item_id', 'user_id'], dtype='object')
Index(['id', 'price'], dtype='object')


Hechemos un vistazo a cómo quedaron los dataframe

In [55]:
user_reviews

,recommend,user_id
0,True,76561197970982479
1,True,76561197970982479
2,True,76561197970982479
3,True,js41637
4,True,js41637
...,...,...
59300,True,76561198312638244
59301,True,76561198312638244
59302,True,LydiaMorley
59303,True,LydiaMorley


In [8]:
user_items

,item_id,user_id
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479
...,...,...
5153204,346330,76561198329548331
5153205,373330,76561198329548331
5153206,388490,76561198329548331
5153207,521570,76561198329548331


In [9]:
steam_games

,id,price
88310,761140,4.99
88311,643980,0.00
88312,670290,0.00
88313,767400,0.99
88315,772540,3.99
...,...,...
120439,745400,1.99
120440,773640,1.99
120441,733530,4.99
120442,610660,1.99


Primero que nada, hago un chequeo de nulos en ambos 3 dataframes

In [10]:
user_items.isnull().sum() # Ninguno, perfecto

item_id    0
user_id    0
dtype: int64

In [11]:
user_reviews.isnull().sum() # Ninguno, perfecto

recommend    0
user_id      0
dtype: int64

In [12]:
steam_games.isnull().sum() # Perfecto

id       0
price    0
dtype: int64

Ahora que ya tengo los dataframe con las columnas que necesito, lo que me resulta más fácil de determinar es cuantos juegos tiene cada usuario. Para ello, utilizo el dataframe user_items

In [13]:
user_items[user_items["user_id"] == "76561197970982479"]["user_id"].count()

277

Primer prototipo del endpoint

In [14]:
def user_data(user_id):
    cantidad_items = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    return cantidad_items

In [15]:
user_data(76561197970982479) # La salida es la esperada

277

En base a esta cantidad de juegos que tiene el usuario, puedo determinar el porcentaje de recomendación del mismo.

In [16]:
user_reviews[(user_reviews["user_id"] == "76561197970982479") & user_reviews["recommend"] == True]["recommend"].count()

3

Segundo prototipo del endpoint

In [17]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    return f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}% de Recomendación"

In [18]:
user_data(76561197970982479)

'1.08% de Recomendación'

Ahora necesito encontrar la cantidad de dinero gastado por el usuario, para ello, tengo que usar los dataframe user_items y steam_games, ya que en ellos se encuentran todos los datos que necesito

In [19]:
steam_games[steam_games["id"] == 10]

,id,price
120416,10,9.99


In [20]:
user_items[user_items["item_id"] == 10] # Tengo que cambiarle el tipo de dato a la columna item_id

,item_id,user_id


In [21]:
user_items["item_id"] = user_items["item_id"].astype(int)

Me doy cuenta que lo más sencillo sería trabajar en un mismo dataframe, por lo cuál decido unir tanto el de items cómo el de games

In [22]:
usuario_juego_precio = pd.merge(user_items, steam_games, how="left", left_on="item_id", right_on="id")
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153204,346330,76561198329548331,346330.0,0.00
5153205,373330,76561198329548331,NaN,NaN
5153206,388490,76561198329548331,388490.0,0.00
5153207,521570,76561198329548331,521570.0,0.00


Voy a tomarme la libertad de eliminar las filas que contienen nulos, ya que de las mismas, no puedo asegurar que el juego es gratuito, ni tampoco que sea pago, así que las elimino

In [23]:
usuario_juego_precio.isnull().sum()

item_id          0
user_id          0
id         1064014
price      1064014
dtype: int64

In [24]:
usuario_juego_precio.dropna(inplace=True)
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153202,304930,76561198329548331,304930.0,0.00
5153203,227940,76561198329548331,227940.0,0.00
5153204,346330,76561198329548331,346330.0,0.00
5153206,388490,76561198329548331,388490.0,0.00


Filtremos el dataset usando el mismo usuario con el cuál veniamos haciendo las pruebas

In [25]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
272,289130,76561197970982479,289130.0,29.99
273,268500,76561197970982479,268500.0,59.99
274,461640,76561197970982479,461640.0,1.99
275,379720,76561197970982479,379720.0,29.99


De aquí, la columna que más nos interesa es la del precio, del cual queremos conocer la suma, la cantidad total gastada por el usuario, así que...

In [26]:
usuario_juego_precio[usuario_juego_precio["user_id"] == "76561197970982479"]["price"].sum() # Ese usuario gastó más de 3400 dólares

3404.3300000000004

Tercer prototipo del endpoint

In [27]:
def user_data(user_id):
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()
    return f"El usuario gastó {round(cantidad_dinero_gastado, 2)} USD"

In [28]:
user_data(76561197970982479)

'El usuario gastó 3404.33 USD'

Cuarto prototipo del endpoint

In [29]:
def user_data(user_id):
    diccionario_de_retorno = {}
    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

    diccionario_de_retorno["Usuario"] = user_id
    diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"
    diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
    diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
    
    return diccionario_de_retorno

In [30]:
user_data(76561197970982479)

{'Usuario': 76561197970982479,
 'Dinero gastado': '3404.33 USD',
 'Porcentaje de recomendación': '1.08%',
 'Cantidad de juegos': 277}

Bueno, se podría decir que ya resolví el endpoint 2, pero en realidad me faltan hacerle algunas aclaraciones y ajustes a la función para que su desempeño sea óptimo

In [32]:
user_data(76561197970982474) # Veo que cuando pongo un id que no corresponde a ningún usuario, me sale lo siguiente

C:\Users\crisr\AppData\Local\Temp\ipykernel_20384\1780711741.py:9: RuntimeWarning: invalid value encountered in scalar divide
  diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"


{'Usuario': 76561197970982474,
 'Dinero gastado': '0.0 USD',
 'Porcentaje de recomendación': 'nan%',
 'Cantidad de juegos': 0}

Quinto prototipo del endpoint

In [34]:
def user_data(user_id):
    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty:
        return "No se encontraron datos para el usuario con id: " + str(user_id)

    cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()
    # cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()
    # cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

    # diccionario_de_retorno["Usuario"] = user_id
    # diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"
    # diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
    # diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
    
    return diccionario_de_retorno

In [35]:
user_data(76561197970982474) # La salida es la esperada.

'No se encontraron datos para el usuario con id: 76561197970982474'

Sexto prototipo de la función

In [36]:
def user_data(user_id):

    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
        return "No se encontraron datos para el usuario con ID: " + str(user_id)
    else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente

        # Se cuenta la cantidad de juegos que tiene el usuario
        cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()

        # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
        cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()

        # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
        cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

        # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
        diccionario_de_retorno["Usuario"] = user_id
        diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"

        # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
        if cantidad_de_recomendaciones == 0:
            diccionario_de_retorno["Porcentaje de recomendaciones"] = "0%"
        else:
            # Si hizo recomendaciones, se hace el calculo normalmente
            diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
        
        diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
        
        # Se devuelve el diccionario creado
        return diccionario_de_retorno

In [37]:
user_data(76561197970982474) # La salida es la esperada.

'No se encontraron datos para el usuario con ID: 76561197970982474'

In [51]:
user_reviews[user_reviews["recommend"] == False] # Probemos con sandwiches1

,recommend,user_id
48,False,76561198043472122
62,False,76561198066046412
68,False,76561198070565427
84,False,boydeer
122,False,sandwiches1
...,...,...
59262,False,kimjongadam
59268,False,iwishihadaids
59283,False,MeloncraftLP
59284,False,76561198277602337


In [53]:
user_reviews[user_reviews["user_id"] == "sandwiches1"] # Ideal, sólo tiene una recomendación y es negativa, osea tiene 0 recomendaciones positivas

,recommend,user_id
122,False,sandwiches1


In [54]:
user_data("sandwiches1")

{'Usuario': 'sandwiches1',
 'Dinero gastado': '1528.9 USD',
 'Porcentaje de recomendaciones': '0%',
 'Cantidad de juegos': 172}

Exporto los dataframe en formato parquet con compresión snappy a la carpeta Datasets para importarlos luego desde allí

In [63]:
user_items.to_parquet("../Datasets/user_items_endpoint_2.parquet",compression="snappy")

In [64]:
user_reviews.to_parquet("../Datasets/user_reviews_endpoint_2.parquet", compression="snappy")

In [65]:
usuario_juego_precio.to_parquet("../Datasets/steam_games_and_users_endpoint_2.parquet", compression="snappy")

Septimo y último prototipo funcional y exportable, le agrego la importación de los archivos correspondientes.

In [ ]:
def user_data(user_id):
    # Cargamos los dataframes dentro de la función
    user_items = pd.read_parquet("./Datasets/user_items_endpoint_2.parquet")
    user_reviews = pd.read_parquet("./Datasets/user_reviews_endpoint_2.parquet")
    usuario_juego_precio = pd.read_parquet("./Datasets/steam_games_and_users_endpoint_2.parquet")

    diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

    if user_items[user_items["user_id"] == str(user_id)].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
        return "No se encontraron datos para el usuario con ID: " + str(user_id)
    else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente

        # Se cuenta la cantidad de juegos que tiene el usuario
        cantidad_juegos = user_items[user_items["user_id"] == str(user_id)]["user_id"].count()

        # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
        cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == str(user_id)) & user_reviews["recommend"] == True]["recommend"].count()

        # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
        cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == str(user_id)]["price"].sum()

        # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
        diccionario_de_retorno["Usuario"] = user_id
        diccionario_de_retorno["Dinero gastado"] = f"{round(cantidad_dinero_gastado, 2)} USD"

        # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
        if cantidad_de_recomendaciones == 0:
            diccionario_de_retorno["Porcentaje de recomendaciones"] = "0%"
        else:
            # Si hizo recomendaciones, se hace el calculo normalmente
            diccionario_de_retorno["Porcentaje de recomendación"] = f"{round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)}%"
        
        diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
        
        # Se devuelve el diccionario creado
        return diccionario_de_retorno

Dado que estoy tratando de ejecutar el endpoint 2 en el main y no estoy podiendo en render debido a errores de memoria, voy a trabajar en optimizar la función

In [ ]:
# El endpoint está así en el main:
from fastapi import FastAPI, HTTPException, Query
import pandas as pd
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
def informacion_usuario(user_id):
    try:
        user_id_str = str(user_id)
        # Cargamos los dataframes dentro de la función
        user_items = pd.read_parquet("../Datasets/user_items_endpoint_2.parquet")
        user_reviews = pd.read_parquet("../Datasets/user_reviews_endpoint_2.parquet")
        usuario_juego_precio = pd.read_parquet("../Datasets/steam_games_and_users_endpoint_2.parquet")
        print("Archivos leídos correctamente")
        
        diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

        if user_items[user_items["user_id"] == user_id_str].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
            return {"Detail":"No se encontraron datos para el usuario con ID: " + user_id_str}
        else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente
            
            # Se cuenta la cantidad de juegos que tiene el usuario
            print("Condicional pasado correctamente")
            cantidad_juegos = user_items[user_items["user_id"] == user_id_str]["user_id"].count()

            print(f"Cantidad de juegos: {cantidad_juegos}, Tipo: {type(cantidad_juegos)}")
            # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
            cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == user_id_str) & (user_reviews["recommend"] == True)]["recommend"].count()
            print("Cantidad de recomendaciones obtenidas")

            # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
            cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == user_id_str]["price"].sum()
            cantidad_dinero_gastado = round(cantidad_dinero_gastado, 2)
            print("Cantidad de dinero gastado obtenido")

            # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
            diccionario_de_retorno["Usuario"] = user_id_str
            print("Usuario agregado al diccionario de salida")

            diccionario_de_retorno["Dinero gastado"] = f"{cantidad_dinero_gastado} USD"
            print("Dinero gastado agregado al diccionario de salida")
            
            # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
            if cantidad_de_recomendaciones == 0:
                porcentaje_recomendacion = 0
                diccionario_de_retorno["Porcentaje de recomendación"] = f"{porcentaje_recomendacion}%"
            else:
                # Si hizo recomendaciones, se hace el calculo normalmente
                porcentaje_recomendacion = round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)
                diccionario_de_retorno["Porcentaje de recomendación"] = f"{porcentaje_recomendacion}%"

            print("Porcentaje de recomendación agregado al diccionario de salida")

            diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
            print("Cantidad de juegos agregados al diccionario de salida")
            
            # Se devuelve el diccionario creado
            return diccionario_de_retorno
        
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=404, detail=f"Archivo Parquet no encontrado: {str(e)}")
    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error al procesar la solicitud: {str(e)}")

Cómo el error es de memoria, voy a trabajar en optimizar los dataframe que estoy cargando dentro del endpoint

In [2]:
import pandas as pd
user_items = pd.read_parquet("../Datasets/user_items_endpoint_2.parquet")
user_reviews = pd.read_parquet("../Datasets/user_reviews_endpoint_2.parquet")
usuario_juego_precio = pd.read_parquet("../Datasets/steam_games_and_users_endpoint_2.parquet")

Comencemos por user_items

In [3]:
user_items 

,item_id,user_id
0,10,76561197970982479
1,20,76561197970982479
2,30,76561197970982479
3,40,76561197970982479
4,50,76561197970982479
...,...,...
5153204,346330,76561198329548331
5153205,373330,76561198329548331
5153206,388490,76561198329548331
5153207,521570,76561198329548331


De este dataframe, bastante grande por cierto (mala mía), para lo único que lo útilizo dentro de la función es para determinar la cantidad de juegos que tiene cada usuario mediante el método .count(). Se me ocurrió que podría ser mucho más optimo que en lugar de tener hacer eso dentro de la función, lo único que se hace sería cargar un dataframe que ya tenga el dato tabular de esta manera: {usuario:cant_juegos}. Así que a ello voy

Acabo de recordar que el archivo original de user items tiene la cantidad de items por usuario junto al user_id, así que para simplificar el trabajo, voy a usar ese archivo:

In [12]:
# Código de "ETL_user_items.ipynb"
import gzip
import ast


rutaArchivo = '../ETL/rawData/users_items.json.gz'

# Lista que va almacenar cada línea del archivo ya descomprimido
data = []

# Metodo open para abrir el archivo comprimido
with gzip.open(rutaArchivo, 'rt', encoding='utf-8') as file:
    # Bucle para iterar el archivo línea por línea
    for line in file:
        try:
            # ast.literal_eval lanza una excepción si la entrada no es un tipo de dato valido en Python. Si lo es, guarda la línea en una variable temporal llamada json_data y luego esa variable temporal es agregada a la lista data previamente creada
            json_data = ast.literal_eval(line)
            data.append(json_data)

        except ValueError as e:
            # Si no se puede leer la línea porque el tipo de dato no es válido en Python, lanza una excepción
            print(f"Error en la línea: {line}")
            continue

# Finalmente, los datos contenidos en la lista data, son alojados en un dataframe de pandas
users_items = pd.DataFrame(data)

In [14]:
users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Dejo sólo las columnas necesarias (las primeras 2)

In [15]:
users_items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [16]:
users_items = users_items[['user_id', 'items_count']]

In [17]:
users_items

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541
...,...,...
88305,76561198323066619,22
88306,76561198326700687,177
88307,XxLaughingJackClown77xX,0
88308,76561198329548331,7


Con este cambio, pase de tener un dataframe de 5.153.209 filas a tener uno de 88310 que me aporta de forma más precisa la información que quiero.
Con esto redujé un 98,2% el Dataframe.
Exporto los cambios hechos.

In [19]:
users_items.to_parquet("../Datasets/endpoint_2/users_items.parquet",compression="snappy")

Continuemos con user_reviews

In [20]:
user_reviews

,recommend,user_id
0,True,76561197970982479
1,True,76561197970982479
2,True,76561197970982479
3,True,js41637
4,True,js41637
...,...,...
59300,True,76561198312638244
59301,True,76561198312638244
59302,True,LydiaMorley
59303,True,LydiaMorley


Este dataframe tiene pocas filas, así que decido conservarlo cómo está

Finalmente, veamos usuario_juego_precio

In [21]:
usuario_juego_precio

,item_id,user_id,id,price
0,10,76561197970982479,10.0,9.99
1,20,76561197970982479,20.0,4.99
2,30,76561197970982479,30.0,4.99
3,40,76561197970982479,40.0,4.99
4,50,76561197970982479,50.0,4.99
...,...,...,...,...
5153202,304930,76561198329548331,304930.0,0.00
5153203,227940,76561198329548331,227940.0,0.00
5153204,346330,76561198329548331,346330.0,0.00
5153206,388490,76561198329548331,388490.0,0.00


Bueno, este dataframe con una cantidad tan enorme de filas lo uso únicamente para determinar la suma de la columna "price", lo cuál me ofrece cómo resultado el dinero total gastado por el usuario. El tema es que para optimizar la función, me conviene tener este dato ya en el mismo Dataframe, y no tener que hacer un sum() a esa columna. Hacemos eso a continuación

In [24]:
user_id = list(usuario_juego_precio["user_id"].unique())
user_id 

['76561197970982479',
 'js41637',
 'evcentric',
 'Riot-Punch',
 'doctr',
 'MinxIsBetterThanPotatoes',
 'NitemarePK',
 'themanwich',
 'maplemage',
 'corrupted_soul',
 'jorellpogi',
 'cadmusthreepointoh',
 '76561198064823719',
 'MeaTCompany',
 '76561198089393905',
 '76561198156664158',
 '76561198077246154',
 '76561198070234207',
 'WeiEDKrSat',
 'thequeenpanda',
 'death-hunter',
 'turneddreamsintoanempire',
 'DJKamBer',
 '76561198030389591',
 'Rainbow-Dashie',
 'btuty8trgt',
 '76561198043472122',
 'MarbleShrine',
 'fdcfelipefdc',
 'bbaekhyun',
 'Moomfina',
 'PPanther',
 'ABSOLUTLY_NOTHING',
 'devvonst',
 'Fr0stedLine',
 '76561198058373434',
 '76561198048353577',
 '76561198072190369',
 '76561198066046412',
 'tru_mu_',
 'wirlom',
 '76561198070565427',
 'SavageEeEe',
 '76561198076909484',
 '76561198066908262',
 'KanadeTheNeko',
 'Leaf_Light_Moscow',
 'erickoiv',
 'GoAwayZach',
 '76561198054540475',
 '76561198100701221',
 '76561198085547365',
 '76561198061252210',
 'OfficialShroomsy',
 '76561

In [28]:
# Librería para ver el progreso del bucle for de acontinuación
from tqdm import tqdm
dinero_gastado_por_usuarios = [] # Lista para almacenar el dinero gastado por cada usuario
for usuario in tqdm(user_id, desc="Procesando usuarios"): # Para cada usuario en la lista user_id creada previamente,
    dinero_local = round(usuario_juego_precio[usuario_juego_precio["user_id"] == usuario]["price"].sum(), 2) # Se crea una variable llamada dinero_local que almacena la cantidad de dinero que tiene el usuario que se esté iterando
    dinero_gastado_por_usuarios.append(dinero_local) # Se agrega la variable local a la lista previamente creada
    º

Procesando usuarios: 100%|██████████| 68590/68590 [3:50:07<00:00,  4.97it/s]   


In [29]:
# A partir de estas ambas listas, voy a crear un dataframe que sea de la forma: usuario | dinero_gastado
usuario_dinero_gastado = pd.DataFrame(
    {"user_id" : user_id,
     "dinero_gastado" : dinero_gastado_por_usuarios}
)
usuario_dinero_gastado

,user_id,dinero_gastado
0,76561197970982479,3404.33
1,js41637,8316.47
2,evcentric,1564.92
3,Riot-Punch,3284.47
4,doctr,6635.61
...,...,...
68585,76561198320136420,673.67
68586,ArkPlays7,134.97
68587,76561198323066619,0.00
68588,76561198326700687,375.83


Bueno, pasé de un dataframe de 4089195 de filas a uno de 68590 filas, logré reducirlo un 98,3% de su tamaño original.
Voy a realizar 3 chequeos para ver si se ejecutó correctamente.

In [30]:
# Chequeo nº1
print(round(usuario_juego_precio[usuario_juego_precio["user_id"] == "76561198329548331"]["price"].sum(), 2)) # Salida esperada: 0.0
print()
# Chequeo nº2
print(round(usuario_juego_precio[usuario_juego_precio["user_id"] == "ArkPlays7"]["price"].sum(), 2)) # Salida esperada: 134.97
print()
# Chequeo nº3
print(round(usuario_juego_precio[usuario_juego_precio["user_id"] == "js41637"]["price"].sum(), 2)) # Salida esperada: 8316.47
print()

0.0

134.97

8316.47



Cómo todo salió correctamente, voy a exportar este dataframe en formato parquet para posteriormente utilizarlo en el main

In [31]:
usuario_dinero_gastado.to_parquet("../Datasets/endpoint_2/usuarios_y_dinero_gastado.parquet",compression="snappy")

A continuación, voy a optimizar el endpoint 2

Octavo prototipo del endpoint

In [37]:
# El endpoint está así en el main:
def informacion_usuario(user_id):
    try:
        user_id_str = str(user_id)
        # Cargamos los dataframes dentro de la función
        user_items = pd.read_parquet("../Datasets/endpoint_2/users_items.parquet")

        user_reviews = pd.read_parquet("../Datasets/endpoint_2/user_reviews_endpoint_2.parquet")

        usuario_juego_precio = pd.read_parquet("../Datasets/endpoint_2/usuarios_y_dinero_gastado.parquet")

        print("Archivos leídos correctamente")
        
        diccionario_de_retorno = {} # Diccionario que va a almacenar las variables correspondientes a la salida de la función

        if user_items[user_items["user_id"] == user_id_str].empty: # En caso de que no haya usuarios con el ID ingresado, se muestra lo siguiente:
            return {"Detail":"No se encontraron datos para el usuario con ID: " + user_id_str}
        else: # En caso de que haya usuarios con el ID ingresado, se muestra lo siguiente
            
            # Se cuenta la cantidad de juegos que tiene el usuario
            print("Condicional pasado correctamente")
            cantidad_juegos = user_items[user_items["user_id"] == user_id_str]["items_count"].iloc[0]

            print(f"Cantidad de juegos: {cantidad_juegos}, Tipo: {type(cantidad_juegos)}")

            # Se cuenta la cantidad de recomendaciones. En la columna recommend aparece "True" o "False", tuve en cuenta sólo las que aparece "True" del usuario ingresado.
            cantidad_de_recomendaciones = user_reviews[(user_reviews["user_id"] == user_id_str) & (user_reviews["recommend"] == True)]["recommend"].count()
            print("Cantidad de recomendaciones obtenidas")
            
            # Se cuenta la cantidad de dinero gastado por el usuario, haciendo una suma de la columna "price" de los juegos que tiene el usuario. 
            cantidad_dinero_gastado = usuario_juego_precio[usuario_juego_precio["user_id"] == user_id_str]["dinero_gastado"].iloc[0]

            cantidad_dinero_gastado = round(cantidad_dinero_gastado, 2)
            print("Cantidad de dinero gastado obtenido")

            # A partir del ID ingresado, del dinero gastado, del porcentaje de recomendación, y de la cantidad de juegos, se almacenan en el diccionario previamente creado con las claves correspondientes.
            diccionario_de_retorno["Usuario"] = user_id_str
            print("Usuario agregado al diccionario de salida")

            diccionario_de_retorno["Dinero gastado"] = f"{cantidad_dinero_gastado} USD"
            print("Dinero gastado agregado al diccionario de salida")
            
            # Si el usuario no hizo recomendaciones, para no tener errores de división por 0, defino que el porcentaje de recomendación es de 0%
            if cantidad_de_recomendaciones == 0:
                porcentaje_recomendacion = 0
                diccionario_de_retorno["Porcentaje de recomendación"] = f"{porcentaje_recomendacion}%"
            else:
                # Si hizo recomendaciones, se hace el calculo normalmente
                porcentaje_recomendacion = round((cantidad_de_recomendaciones * 100) / cantidad_juegos, 2)
                diccionario_de_retorno["Porcentaje de recomendación"] = f"{porcentaje_recomendacion}%"

            print("Porcentaje de recomendación agregado al diccionario de salida")

            diccionario_de_retorno["Cantidad de juegos"] = cantidad_juegos
            print("Cantidad de juegos agregados al diccionario de salida")
            
            # Se devuelve el diccionario creado
            return diccionario_de_retorno
        
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=404, detail=f"Archivo Parquet no encontrado: {str(e)}")
    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Error al procesar la solicitud: {str(e)}")

In [38]:
informacion_usuario("sandwiches1")

Archivos leídos correctamente
Condicional pasado correctamente
Cantidad de juegos: 172, Tipo: <class 'numpy.int64'>
Cantidad de recomendaciones obtenidas
Cantidad de dinero gastado obtenido
Usuario agregado al diccionario de salida
Dinero gastado agregado al diccionario de salida
Porcentaje de recomendación agregado al diccionario de salida
Cantidad de juegos agregados al diccionario de salida


{'Usuario': 'sandwiches1',
 'Dinero gastado': '1528.9 USD',
 'Porcentaje de recomendación': '0%',
 'Cantidad de juegos': 172}